# Pre-processing California Allocation data for WaDEQA upload.
Date Updated: 07/21/2021
Purpose:  To pre-process the California data into one master file for simple DataFrame creation and extraction

Notes:
- asdf

In [1]:
#Needed Libararies
import os
import numpy as np
import pandas as pd
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

In [2]:
#Working Directory and Input File
workingDir = "C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/California/WaterAllocation/RawInputData"
os.chdir(workingDir)

fileInput1 = "EWRIMS MASTER FLAT FILE DATA DICTIONARY DRAFT 1-17-20.xlsx" # date & beneficial use data
fileInput2 = "POD_Attributes_input.csv" # wr data
fileInput3 = "Points_of_Diversion_20210701_input.csv" # site data

df_emff = pd.read_excel(fileInput1, header=0, sheet_name="ewrims_flat_file", skiprows=0).replace(np.nan, "")
df_poda = pd.read_csv(fileInput2).replace(np.nan, "")
df_point = pd.read_csv(fileInput3).replace(np.nan, "")

In [3]:
# Merging dataframes into one, using left-join.
dfPOD = pd.merge(df_poda, df_point, left_on='CORE_POD_ID', right_on='POD_ID', how='left')
dfPOD = pd.merge(dfPOD, df_emff[['WR_WATER_RIGHT_ID', 'PRIORITY_DATE', 'DIRECT_DIV_SEASON_END', 'DIRECT_DIV_SEASON_START', 'APPLICATION_ACCEPTANCE_DATE', 'USE_CODE']], on='WR_WATER_RIGHT_ID', how='left')
print(len(dfPOD))
dfPOD.head(3)

63900


,OID__x,CORE_POD_ID,WR_WATER_RIGHT_ID,POD_NUMBER,APPL_ID_x,APPL_POD_x,DIRECT_DIV_AMOUNT,DIVERSION_STORAGE_AMOUNT,DIVERSION_AC_FT,P_PLACE_ID,POD_STATUS,FACE_VALUE_AMOUNT,DIVERSION_TYPE,DIVERSION_CODE_TYPE,WR_TYPE,WR_STATUS,STORAGE_TYPE,POD_UNIT,FIRST_NAME,LAST_NAME,ENTITY_TYPE,OID__y,POD_ID,APPL_ID_y,POD_NUM,APPL_POD_y,TOWNSHIP_NUMBER,TOWNSHIP_DIRECTION,RANGE_NUMBER,RANGE_DIRECTION,SECTION_NUMBER,SECTION_CLASSIFIER,QUARTER,QUARTER_QUARTER,MERIDIAN,NORTH_COORD,EAST_COORD,SP_ZONE,LATITUDE,LONGITUDE,TRIB_DESC,LOCATION_METHOD,SOURCE_NAME,MOVEABLE,HAS_OPOD,WATERSHED,COUNTY,WELL_NUMBER,QUAD_MAP_NAME,QUAD_MAP_NUM,QUAD_MAP_MIN_SER,PARCEL_NUMBER,DIVERSION_SITE_NAME,LAST_UPDATE_DATE,LAST_UPDATE_USER_ID,SPECIAL_AREA,HUC_12,HUC_8,HU_8_NAME,HU_12_NAME,PRIORITY_DATE,DIRECT_DIV_SEASON_END,DIRECT_DIV_SEASON_START,APPLICATION_ACCEPTANCE_DATE,USE_CODE
0,1,14949,36668,01,S013799,S013799_01,0.066,0.0,0.0,669953,Active,0.0,Diversion to Storage,Diversion point,Statement of Div and Use,Claimed,Point of Diversion,Cubic Feet per Second,CARL,SHIMEL,Individual,91156.0,14949.0,S013799,01,S013799_01,33.0,N,11.0,W,1.0,,,,21.0,2154480.547973,6265407.942305,1.0,40.739495,-123.069210,,DD_NE,HOCKER GULCH,N,Y,TRINITY RIVER,Trinity,,JUNCTION CITY,CC012,7.5,012-260-70,,9/29/2007 6:14:12,9.0,,1.801021e+11,18010211.0,Trinity,Conner Creek Trinity River,,2020-10-31 00:00:00,2020-06-01 00:00:00,1991-12-30 00:00:00,Irrigation
1,2,14950,36694,01,S013825,S013825_01,192.0,0.0,0.2,669954,Active,0.0,Diversion to Storage,Diversion point,Statement of Div and Use,Claimed,Point of Diversion,Gallons per Day,SHERBURNE,SLACK,Individual,67266.0,14950.0,S013825,01,S013825_01,8.0,S,4.0,W,4.0,,NE,NW,21.0,1926358.638086,6035514.543569,3.0,37.271356,-122.308314,,DD_NE,PESCADERO CREEK,N,N,SAN MATEO,San Mateo,,LA HONDA,J 041,7.5,,,9/29/2007 6:13:58,9.0,,1.805001e+11,18050006.0,San Francisco Coastal South,Lower Pescadero Creek,,2020-12-31 00:00:00,2020-01-01 00:00:00,1992-08-28 00:00:00,Domestic
2,3,14951,2273,01,A010397,A010397_01,200.0,0.0,0.0,669955,Active,0.1,Diversion to Storage,Diversion point,Appropriative,Licensed,Point of Diversion,Gallons per Day,AL,RENKER,Individual,97853.0,14951.0,A010397,01,A010397_01,11.0,N,15.0,E,27.0,,NW,NE,21.0,2047785.971346,7050062.4187,2.0,38.772588,-120.286795,,DD_NE,SOUTH FORK AMERICAN RIVER,N,N,AMERICAN RIVER,El Dorado,,KYBURZ,G 061,7.5,012-220-03-01,,9/29/2007 6:14:05,9.0,DELTA WATERSHED,1.802013e+11,18020129.0,South Fork American,Chimney Flat-South Fork American River,,2020-11-01 00:00:00,2020-04-01 00:00:00,1942-03-11 00:00:00,Domestic


In [4]:
print("Allocation priority date...")

def formatDateString(inString1):
    inString = str(inString1).strip()
    try:
        if inString == '' or pd.isnull(inString):
            valndf = ''
        else:
            valD = pd.to_datetime(inString) # Also valD = parse(inString) #--datetuil.parser.parse
            valnDd = valD.date()
            valndf = valnDd.strftime('%m/%d/%Y')
    except:
        valndf = ''
    return valndf

dfPOD['in_AllocationPriorityDate'] = dfPOD.apply(lambda row: formatDateString(row['PRIORITY_DATE']) if str(row['PRIORITY_DATE']) != ''else formatDateString(row['APPLICATION_ACCEPTANCE_DATE']), axis=1)
dfPOD.head(3)

Allocation priority date...


,OID__x,CORE_POD_ID,WR_WATER_RIGHT_ID,POD_NUMBER,APPL_ID_x,APPL_POD_x,DIRECT_DIV_AMOUNT,DIVERSION_STORAGE_AMOUNT,DIVERSION_AC_FT,P_PLACE_ID,POD_STATUS,FACE_VALUE_AMOUNT,DIVERSION_TYPE,DIVERSION_CODE_TYPE,WR_TYPE,WR_STATUS,STORAGE_TYPE,POD_UNIT,FIRST_NAME,LAST_NAME,ENTITY_TYPE,OID__y,POD_ID,APPL_ID_y,POD_NUM,APPL_POD_y,TOWNSHIP_NUMBER,TOWNSHIP_DIRECTION,RANGE_NUMBER,RANGE_DIRECTION,SECTION_NUMBER,SECTION_CLASSIFIER,QUARTER,QUARTER_QUARTER,MERIDIAN,NORTH_COORD,EAST_COORD,SP_ZONE,LATITUDE,LONGITUDE,TRIB_DESC,LOCATION_METHOD,SOURCE_NAME,MOVEABLE,HAS_OPOD,WATERSHED,COUNTY,WELL_NUMBER,QUAD_MAP_NAME,QUAD_MAP_NUM,QUAD_MAP_MIN_SER,PARCEL_NUMBER,DIVERSION_SITE_NAME,LAST_UPDATE_DATE,LAST_UPDATE_USER_ID,SPECIAL_AREA,HUC_12,HUC_8,HU_8_NAME,HU_12_NAME,PRIORITY_DATE,DIRECT_DIV_SEASON_END,DIRECT_DIV_SEASON_START,APPLICATION_ACCEPTANCE_DATE,USE_CODE,in_AllocationPriorityDate
0,1,14949,36668,01,S013799,S013799_01,0.066,0.0,0.0,669953,Active,0.0,Diversion to Storage,Diversion point,Statement of Div and Use,Claimed,Point of Diversion,Cubic Feet per Second,CARL,SHIMEL,Individual,91156.0,14949.0,S013799,01,S013799_01,33.0,N,11.0,W,1.0,,,,21.0,2154480.547973,6265407.942305,1.0,40.739495,-123.069210,,DD_NE,HOCKER GULCH,N,Y,TRINITY RIVER,Trinity,,JUNCTION CITY,CC012,7.5,012-260-70,,9/29/2007 6:14:12,9.0,,1.801021e+11,18010211.0,Trinity,Conner Creek Trinity River,,2020-10-31 00:00:00,2020-06-01 00:00:00,1991-12-30 00:00:00,Irrigation,12/30/1991
1,2,14950,36694,01,S013825,S013825_01,192.0,0.0,0.2,669954,Active,0.0,Diversion to Storage,Diversion point,Statement of Div and Use,Claimed,Point of Diversion,Gallons per Day,SHERBURNE,SLACK,Individual,67266.0,14950.0,S013825,01,S013825_01,8.0,S,4.0,W,4.0,,NE,NW,21.0,1926358.638086,6035514.543569,3.0,37.271356,-122.308314,,DD_NE,PESCADERO CREEK,N,N,SAN MATEO,San Mateo,,LA HONDA,J 041,7.5,,,9/29/2007 6:13:58,9.0,,1.805001e+11,18050006.0,San Francisco Coastal South,Lower Pescadero Creek,,2020-12-31 00:00:00,2020-01-01 00:00:00,1992-08-28 00:00:00,Domestic,08/28/1992
2,3,14951,2273,01,A010397,A010397_01,200.0,0.0,0.0,669955,Active,0.1,Diversion to Storage,Diversion point,Appropriative,Licensed,Point of Diversion,Gallons per Day,AL,RENKER,Individual,97853.0,14951.0,A010397,01,A010397_01,11.0,N,15.0,E,27.0,,NW,NE,21.0,2047785.971346,7050062.4187,2.0,38.772588,-120.286795,,DD_NE,SOUTH FORK AMERICAN RIVER,N,N,AMERICAN RIVER,El Dorado,,KYBURZ,G 061,7.5,012-220-03-01,,9/29/2007 6:14:05,9.0,DELTA WATERSHED,1.802013e+11,18020129.0,South Fork American,Chimney Flat-South Fork American River,,2020-11-01 00:00:00,2020-04-01 00:00:00,1942-03-11 00:00:00,Domestic,03/11/1942


In [5]:
print("Timeframe start and time frame end...")

def formatDateString2(inString1):
    inString = str(inString1).strip()
    try:
        if inString == '' or pd.isnull(inString):
            valndf = ''
        else:            
            valD = pd.to_datetime(inString)
            valnDd = valD.date()
            valndf = valnDd.strftime('%m/%d')
    except:
        valndf = ''
    return valndf

dfPOD['in_AllocationTimeframeStart'] = dfPOD.apply(lambda row: formatDateString2(row['DIRECT_DIV_SEASON_START']), axis=1)
dfPOD['in_AllocationTimeframeEnd'] = dfPOD.apply(lambda row: formatDateString2(row['DIRECT_DIV_SEASON_END']), axis=1)
dfPOD.head(3)

Timeframe start and time frame end...


,OID__x,CORE_POD_ID,WR_WATER_RIGHT_ID,POD_NUMBER,APPL_ID_x,APPL_POD_x,DIRECT_DIV_AMOUNT,DIVERSION_STORAGE_AMOUNT,DIVERSION_AC_FT,P_PLACE_ID,POD_STATUS,FACE_VALUE_AMOUNT,DIVERSION_TYPE,DIVERSION_CODE_TYPE,WR_TYPE,WR_STATUS,STORAGE_TYPE,POD_UNIT,FIRST_NAME,LAST_NAME,ENTITY_TYPE,OID__y,POD_ID,APPL_ID_y,POD_NUM,APPL_POD_y,TOWNSHIP_NUMBER,TOWNSHIP_DIRECTION,RANGE_NUMBER,RANGE_DIRECTION,SECTION_NUMBER,SECTION_CLASSIFIER,QUARTER,QUARTER_QUARTER,MERIDIAN,NORTH_COORD,EAST_COORD,SP_ZONE,LATITUDE,LONGITUDE,TRIB_DESC,LOCATION_METHOD,SOURCE_NAME,MOVEABLE,HAS_OPOD,WATERSHED,COUNTY,WELL_NUMBER,QUAD_MAP_NAME,QUAD_MAP_NUM,QUAD_MAP_MIN_SER,PARCEL_NUMBER,DIVERSION_SITE_NAME,LAST_UPDATE_DATE,LAST_UPDATE_USER_ID,SPECIAL_AREA,HUC_12,HUC_8,HU_8_NAME,HU_12_NAME,PRIORITY_DATE,DIRECT_DIV_SEASON_END,DIRECT_DIV_SEASON_START,APPLICATION_ACCEPTANCE_DATE,USE_CODE,in_AllocationPriorityDate,in_AllocationTimeframeStart,in_AllocationTimeframeEnd
0,1,14949,36668,01,S013799,S013799_01,0.066,0.0,0.0,669953,Active,0.0,Diversion to Storage,Diversion point,Statement of Div and Use,Claimed,Point of Diversion,Cubic Feet per Second,CARL,SHIMEL,Individual,91156.0,14949.0,S013799,01,S013799_01,33.0,N,11.0,W,1.0,,,,21.0,2154480.547973,6265407.942305,1.0,40.739495,-123.069210,,DD_NE,HOCKER GULCH,N,Y,TRINITY RIVER,Trinity,,JUNCTION CITY,CC012,7.5,012-260-70,,9/29/2007 6:14:12,9.0,,1.801021e+11,18010211.0,Trinity,Conner Creek Trinity River,,2020-10-31 00:00:00,2020-06-01 00:00:00,1991-12-30 00:00:00,Irrigation,12/30/1991,06/01,10/31
1,2,14950,36694,01,S013825,S013825_01,192.0,0.0,0.2,669954,Active,0.0,Diversion to Storage,Diversion point,Statement of Div and Use,Claimed,Point of Diversion,Gallons per Day,SHERBURNE,SLACK,Individual,67266.0,14950.0,S013825,01,S013825_01,8.0,S,4.0,W,4.0,,NE,NW,21.0,1926358.638086,6035514.543569,3.0,37.271356,-122.308314,,DD_NE,PESCADERO CREEK,N,N,SAN MATEO,San Mateo,,LA HONDA,J 041,7.5,,,9/29/2007 6:13:58,9.0,,1.805001e+11,18050006.0,San Francisco Coastal South,Lower Pescadero Creek,,2020-12-31 00:00:00,2020-01-01 00:00:00,1992-08-28 00:00:00,Domestic,08/28/1992,01/01,12/31
2,3,14951,2273,01,A010397,A010397_01,200.0,0.0,0.0,669955,Active,0.1,Diversion to Storage,Diversion point,Appropriative,Licensed,Point of Diversion,Gallons per Day,AL,RENKER,Individual,97853.0,14951.0,A010397,01,A010397_01,11.0,N,15.0,E,27.0,,NW,NE,21.0,2047785.971346,7050062.4187,2.0,38.772588,-120.286795,,DD_NE,SOUTH FORK AMERICAN RIVER,N,N,AMERICAN RIVER,El Dorado,,KYBURZ,G 061,7.5,012-220-03-01,,9/29/2007 6:14:05,9.0,DELTA WATERSHED,1.802013e+11,18020129.0,South Fork American,Chimney Flat-South Fork American River,,2020-11-01 00:00:00,2020-04-01 00:00:00,1942-03-11 00:00:00,Domestic,03/11/1942,04/01,11/01


In [6]:
#Creating WaDE Owner Field.  
#Create from Owner field. If empty, use LastName + FirstName fields.

def retrieveOwner(FN, LN):
    FN = str(FN).strip()
    LN = str(LN).strip()
    outList = FN + " " + LN
    if FN == "" or pd.isnull(FN):
        outList = LN
    if LN == "" or pd.isnull(LN):
        outList = FN
    if (FN == "" and LN == "") or (pd.isnull(FN) and pd.isnull(LN)):
        outList = "Unspecified"
    print(outList)
    return outList

dfPOD['in_WaDEOwner'] = dfPOD.apply(lambda row: retrieveOwner(row['FIRST_NAME'], row['LAST_NAME']), axis=1)
dfPOD.head(3)

CARL SHIMEL
SHERBURNE SLACK
AL RENKER
MARY CHARTER
U S SHASTA-TRINITY NATL FOREST
SHIRLEY SMITH
US KLAMATH NATL FOREST
JACKI MATEJKA
THE VANDER WOUDE FAMILY REVOCABLE TRUST OF NOV 25 2003
WYNN MYERS
RICHARD JENNINGS
LYALL ENTERPRISES INC
STATE WATER RESOURCES CONTROL BOARD
CECELIA MULLEN
PAUL NIPPERT
JACKSON VALLEY IRRIGATION DISTRICT
WARWICK MOBILE HOME PARK LLC
HORACE MEYER ESTATE
CALAVERAS MURPHY PROPERTY LP
MAINE PRAIRIE WATER DISTRICT
MAINE PRAIRIE WATER DISTRICT
MICHAEL BROWN
TAMARACK LODGE LLC
MUIR TRAIL RANCH, INC
MERCED FALLS INVESTORS LLC
MERCED FALLS INVESTORS LLC
MERCED FALLS INVESTORS LLC
ROBERT H MACKEY AND SONS INC
LOIS STAMPER
KIBBIE RUTH
E MARTIN KERNS
DANIEL MCQUEENEY
CARPENTER FAMILY TRUST
JEFF CHANCE
JOHN SMALLEY
CALIF DEPT OF FORESTRY & FIRE PROTECTION
Unspecified
TREASURY WINE ESTATES
NELDA M. FRAGUERO TRUSTEE OF NELDA M. FRAGUERO 2013 REVOCABLE TRUST
U S BUREAU OF LAND MANAGEMENT
RICHARD SCHWENDINGER
EDNA LEARNED
THE BRUCE & PAT BLAIKIE FAMILY TRUST DATED 5/20/19

SONOMA COUNTY WATER AGENCY
LYNETTE ROSE
JENNIFER GREENE
DINO ANDREOTTI
JOEY RATTO JR
LAKESIDE PARK ASSOCIATION
RAYMOND SOLBAU
William Jamison
William Jamison
BORGES RANCH LLC
ANDREW ZLOT
Unspecified
Unspecified
SAN MIGUELITO MUTUAL WATER COMPANY
DENNIS WRIGHT
Anthony & Lura Vieira
BEN WANAT
JEFFERY BRYANT
ELBAR INVESTMENTS L P
HAROLD DULL
RICHARD JENNINGS
BECKSTOFFER VINEYARDS XVI
BECKSTOFFER VINEYARDS XVI
BECKSTOFFER VINEYARDS XVI
MENDIBOURE RANCH INC
MENDIBOURE RANCH INC
MENDIBOURE RANCH INC
MENDIBOURE RANCH INC
MENDIBOURE RANCH INC
MENDIBOURE RANCH INC
MENDIBOURE RANCH INC
MENDIBOURE RANCH INC
ESTILL RANCHES LLC
KEVIN KINSELLA
EL DORADO IRRIGATION DISTRICT
EL DORADO IRRIGATION DISTRICT
USIBELLI RANCH LLC
M-R VINEYARD
LOIS DUNKIN-POTTS
DONALD SORENSON
HIDDEN VALLEY LAKE COMMUNITY SERVICES DISTRICT
HIDDEN VALLEY LAKE COMMUNITY SERVICES DISTRICT
HIDDEN VALLEY LAKE COMMUNITY SERVICES DISTRICT
SAN BERNABE VINEYARDS
PAUL DEUTSCHMAN
PAUL DEUTSCHMAN
US BUREAU OF LAND MANAGEMENT
US BUREAU OF

US BUREAU OF LAND MANAGEMENT
US BUREAU OF LAND MANAGEMENT
US BUREAU OF LAND MANAGEMENT
US BUREAU OF LAND MANAGEMENT
US BUREAU OF LAND MANAGEMENT, UKIAH FIELD OFFICE
US BUREAU OF LAND MANAGEMENT
U S NATIONAL PARK SERVICE
US BUREAU OF LAND MANAGEMENT
US BUREAU OF LAND MANAGEMENT
US BUREAU OF LAND MANAGEMENT
US BUREAU OF LAND MANAGEMENT
CAMILLE BELCHER HARRISON
DUANE FITTERER
E.W. Merritt Farms
E.W. Merritt Farms
E.W. Merritt Farms
E.W. Merritt Farms
R HAMBY
FARMERS DITCH COMPANY
RALPH MEHRTEN
Unspecified
TONY PEJU
Borri Ranch
Alexis Woods
DUFF BEVILL
DUFF BEVILL
DUFF BEVILL
EL DORADO COUNTY WATER AGENCY
JOHN HANCOCK LIFE INSURANCE COMPANY USA
JOHN HANCOCK LIFE INSURANCE COMPANY USA
Dry Creek Vineyard Inc
Dry Creek Vineyard Inc
HANS FAHDEN VINEYARDS
Jeff Manker
USDA Forest Service-Modoc National Forest
Andreas Baer
CECILIA REYNOLDS
The Hess Collection Winery, Veeder Hills Ranch
Kathleen Andrews
Paul Tarushka
TIMOTHY SKALLAND
KARL KRIEDT
WILLIAM GERMAN
James Serviolo
COOLEY RANCH COMPANY -

BH Farming
BH Farming
CORONA PROJECT, LLC
U S LASSEN NATL FOREST
FRANCISCAN VINEYARDS INC
CAMBRIA COMMUNITY SERVICES DISTRICT
FRANCISCAN VINEYARDS INC
U S MODOC NATL FOREST
SUSIE MOODY
SUSIE MOODY
MICHAEL CREBBIN
Unspecified
JOHN BAKER
HECTOR CORNELIO
Unspecified
GILMAN ORDWAY
GILMAN ORDWAY
GILMAN ORDWAY
GILMAN ORDWAY
Unspecified
Unspecified
CALIF DEPT OF PARKS & RECREATION
HOWELL MOUNTAIN MUTUAL WATER CO, INC
ECCO WATER COMPANY
THOMAS AND SHARON WALDEMER FAMILY TRUST
ENJ PROPERTIES LLC
Unspecified
CAROL FARROW
EL DORADO COUNTY WATER AGENCY
LEO KENCHELOE
RONALD KETLER
FROSTY ACRES INCORPORATED
MARVIN PETERSON
WILLOW CREEK GOLF & COUNTRY CLUB
WILLOW CREEK GOLF & COUNTRY CLUB
JOSEPH MONTANA JR
DIAMOND MOUNTAIN VINEYARD COMPANY
CURTIS FAMILY TRUST
RICHARD ROBERTS
GABILAN CATTLE COMPANY
GABILAN CATTLE COMPANY
GABILAN CATTLE COMPANY
GABILAN CATTLE COMPANY
ISHI PISHI CLUB
Unspecified
Unspecified
Unspecified
STEPHEN ADAMS
SCOTT BROWN
DOUG HILL
ANNA NICKERSON
GERALD JOHNS
DONALD FERGUSON
SC RA

LAWRENCE STORNETTA
CALIF DEPT OF PARKS & RECREATION
William Jamison
William Jamison
STEPHANIE MILLS
CALIFORNIA DEPARTMENT OF FISH AND WILDLIFE
USDA FOREST SERVICE LAKE TAHOE BASIN MANAGEMENT UNIT
U S SHASTA-TRINITY NATL FOREST
CALIF DEPT OF PARKS & RECREATION
U S MODOC NATL FOREST
STEVE KOPF TRUST
PHYLLIS BAILEY
JOE HOLGUIN
JOE HOLGUIN
HEIDRICK FAMILY TRUST
U S FOREST SERVICE
CITY OF EUREKA
CITY OF EUREKA
CITY OF EUREKA
LANCE BARLEAN
Unspecified
Unspecified
VALLEY OF ENCHANTMENT MUTUAL WATER CO
Unspecified
US KLAMATH NATL FOREST
ALBERT CASH
STATE WATER RESOURCES CONTROL BOARD
WILLIE KOOLSTRA
CALAVERAS COUNTY WATER DISTRICT
LEMA 1992 TRUST
GENE BOWER
VAN GRONINGEN & SONS INC
VAN GRONINGEN & SONS INC
JACK COWLEY FAMILY RANCH
U S MODOC NATL FOREST
U S MODOC NATL FOREST
ISAAC ABRAMS
LEDBETTER FARMS INC
LEDBETTER FARMS INC
CAMP TAYLOR, INC.
PATIANNA ORGANIC VINEYARDS
PATIANNA ORGANIC VINEYARDS
REINOSO PEAK LAND AND CATTLE CO LLC
REINOSO PEAK LAND AND CATTLE CO LLC
ANITA MERLO
Riverwest Proc

...

PROPERTY RESERVE CALIFORNIA, INC
PROPERTY RESERVE CALIFORNIA, INC
PROPERTY RESERVE CALIFORNIA, INC
PROPERTY RESERVE CALIFORNIA, INC
CALIF DEPT OF TRANSPORTATION
GUERRA ORCHARD
MAXWELL IRRIGATION DISTRICT
MAXWELL IRRIGATION DISTRICT
USDA LOS PADRES NATIONAL FOREST
RAYMOND FUREY
CALIF DEPT OF PARKS & RECREATION
US BUREAU OF LAND MANAGEMENT
U S FOREST SERVICE
PacificUS Real Estate Group
LYLE TURPIN
MONTEREY COUNTY WATER RESOURCES AGENCY
THE RANDALL A WOLF TRUST, UTA JUNE 1, 1984
THE RANDALL A WOLF TRUST, UTA JUNE 1, 1984
WEI HONG
Michael Bonnheim
GEORGE SULLIVAN
GALLO VINEYARDS INC
RAMON HERNANDEZ
FRITZ WINERY
MOUNT ST HELENA VINEYARDS
JOHN BULLOCK
JOSE REYNOSO
ROY ARNTSON
AGRI-EMPIRE, INC
AGRI-EMPIRE, INC
AGRI-EMPIRE, INC
Ailish Schutz
ANDERSON VINEYARDS INC
MITCHELL PORTERA
MITCHELL PORTERA
MITCHELL PORTERA
2003 MARINO FAMILY TRUST AND CHRISTOPHER JOSEPH HARNEY AND CHRISTINA MARINO HARNEY REV LIVING TRUST
SONNY LLC
ROBERT LISSNER
BERNARD LACROUTE
TERRY CONROY
PEARSON RANCH
MARY SMITH
EL

Dry Lake Properties, LLC
Dry Lake Properties, LLC
YORK CREEK VINEYARDS
HARRIS RANCH LLC
ROGER VAN CRAEYNEST
ALEJANDRO MEDINA
LAURA BRANDON
TRINITY CENTER MUTUAL WATER COMPANY
STEPHEN DOMENIK
US BUREAU OF LAND MANAGEMENT
WILLIAM H DAUGHERTY IRREVOCABLE TRUST FBO G W & J M DAUGHERTY
SAMUEL EAKLE
GREGORY BEEKEN
SCOTT BROWN
BAY TREE VINEYARDS, L.L.C.
MARGARET M NOBMANN FAMILY TRUST
SIERRA PACIFIC INDUSTRIES
DIAMOND LANDS CORPORATION
DIAMOND LANDS CORPORATION
SAN GABRIEL VALLEY PROTECTIVE  ASSOCIATION
JOHN C AND CATHERINE L MUNDT TRUSTEES
JAMES TAYLOR
GOLDEN STATE WATER COMPANY
U S ELDORADO NATL FOREST
USDA LOS PADRES NATIONAL FOREST
MICHAEL VISMAN
SUVIK FARMS, LLC
RAFAEL REGALADO
U S INYO NATL FOREST
RICHARD CARTER
TISDALE IRRIGATION & DRAINAGE CO
TISDALE IRRIGATION & DRAINAGE CO
Brian Johnson
KENNETH LAIRD
STATE WATER RESOURCES CONTROL BOARD
STATE WATER RESOURCES CONTROL BOARD
PAUL WATTIS JR
PAUL WATTIS JR
PAUL WATTIS JR
DORIC JEMISON-BALL II
ROBERT HOKANSON
RICHARD RIOLO
DOUGLAS KORTEN
U

GIRL SCOUTS OF GREATER LOS ANGELES
GIRL SCOUTS OF GREATER LOS ANGELES
U S MENDOCINO NATL FOREST
ELLEN MURPHY-WELK
SISKIYOU COUNTY BOARD OF EDUCATION
JULIANA MUTUAL WATER COMPANY
U S NATIONAL PARK SERVICE
CALIF DEPT OF WATER RESOURCES
WALTER SILACCI
Samuel Cohen
John G Reinecke Trust
FRED SCHLUETER
CALIF DEPT OF WATER RESOURCES
CALIF DEPT OF WATER RESOURCES
CALIF DEPT OF WATER RESOURCES
CALIF DEPT OF WATER RESOURCES
CALIF DEPT OF WATER RESOURCES
YUBA COUNTY WATER AGENCY
STATE WATER RESOURCES CONTROL BOARD
STATE WATER RESOURCES CONTROL BOARD
STATE WATER RESOURCES CONTROL BOARD
STATE WATER RESOURCES CONTROL BOARD
AMADOR WATER AGENCY
Jefferey Straubel
CAYUCOS BEACH MUTUAL WATER COMPANY
Katherine Royce-Salazar
US SAN BERNARDINO NATL FOREST
CA CRAWFORD PROPERTIES LLC
TIM WEST
SKYLAR WALKER
WEST COAST VENEER COMPANY
KERO-TV
ERICH SENDER
PHILLIP WASSON
PHILLIP WASSON
PHILLIP WASSON
PHILLIP WASSON
STATE WATER RESOURCES CONTROL BOARD
U S MODOC NATL FOREST
POCKET RANCH LLC
CAMERON SPAULDING
GARDI

US BUREAU OF LAND MANAGEMENT
US BUREAU OF LAND MANAGEMENT
WILLIAM WISE
US BUREAU OF LAND MANAGEMENT
US BUREAU OF LAND MANAGEMENT
US BUREAU OF LAND MANAGEMENT
LOS ANGELES DEPARTMENT OF WATER AND POWER
LOS ANGELES DEPARTMENT OF WATER AND POWER
LOS ANGELES DEPARTMENT OF WATER AND POWER
LOS ANGELES DEPARTMENT OF WATER AND POWER
LOS ANGELES DEPARTMENT OF WATER AND POWER
LOS ANGELES DEPARTMENT OF WATER AND POWER
LOS ANGELES DEPARTMENT OF WATER AND POWER
BAR 65 COMPANY
C JOHNSON
JACK JONES
SOUTHERN CALIFORNIA EDISON COMPANY
SOUTHERN CALIFORNIA EDISON COMPANY
CHARLES GIOVANNETTI
Unspecified
U S MODOC NATL FOREST
U S MODOC NATL FOREST
U S MODOC NATL FOREST
U S MODOC NATL FOREST
US KLAMATH NATL FOREST
US KLAMATH NATL FOREST
U S TOIYABE NATL FOREST
US BUREAU OF LAND MANAGEMENT / PALM SPRINGS FIELD OFFICE
U S TOIYABE NATL FOREST
U S TOIYABE NATL FOREST
FRANCIS CARRINGTON
VIKING RANCH
VIKING RANCH
STINSON CANAL & IRRIGATION CO
WESTERN CANAL WATER DISTRICT
US BUREAU OF LAND MANAGEMENT
US BUREAU OF L

,OID__x,CORE_POD_ID,WR_WATER_RIGHT_ID,POD_NUMBER,APPL_ID_x,APPL_POD_x,DIRECT_DIV_AMOUNT,DIVERSION_STORAGE_AMOUNT,DIVERSION_AC_FT,P_PLACE_ID,POD_STATUS,FACE_VALUE_AMOUNT,DIVERSION_TYPE,DIVERSION_CODE_TYPE,WR_TYPE,WR_STATUS,STORAGE_TYPE,POD_UNIT,FIRST_NAME,LAST_NAME,ENTITY_TYPE,OID__y,POD_ID,APPL_ID_y,POD_NUM,APPL_POD_y,TOWNSHIP_NUMBER,TOWNSHIP_DIRECTION,RANGE_NUMBER,RANGE_DIRECTION,SECTION_NUMBER,SECTION_CLASSIFIER,QUARTER,QUARTER_QUARTER,MERIDIAN,NORTH_COORD,EAST_COORD,SP_ZONE,LATITUDE,LONGITUDE,TRIB_DESC,LOCATION_METHOD,SOURCE_NAME,MOVEABLE,HAS_OPOD,WATERSHED,COUNTY,WELL_NUMBER,QUAD_MAP_NAME,QUAD_MAP_NUM,QUAD_MAP_MIN_SER,PARCEL_NUMBER,DIVERSION_SITE_NAME,LAST_UPDATE_DATE,LAST_UPDATE_USER_ID,SPECIAL_AREA,HUC_12,HUC_8,HU_8_NAME,HU_12_NAME,PRIORITY_DATE,DIRECT_DIV_SEASON_END,DIRECT_DIV_SEASON_START,APPLICATION_ACCEPTANCE_DATE,USE_CODE,in_AllocationPriorityDate,in_AllocationTimeframeStart,in_AllocationTimeframeEnd,in_WaDEOwner
0,1,14949,36668,01,S013799,S013799_01,0.066,0.0,0.0,669953,Active,0.0,Diversion to Storage,Diversion point,Statement of Div and Use,Claimed,Point of Diversion,Cubic Feet per Second,CARL,SHIMEL,Individual,91156.0,14949.0,S013799,01,S013799_01,33.0,N,11.0,W,1.0,,,,21.0,2154480.547973,6265407.942305,1.0,40.739495,-123.069210,,DD_NE,HOCKER GULCH,N,Y,TRINITY RIVER,Trinity,,JUNCTION CITY,CC012,7.5,012-260-70,,9/29/2007 6:14:12,9.0,,1.801021e+11,18010211.0,Trinity,Conner Creek Trinity River,,2020-10-31 00:00:00,2020-06-01 00:00:00,1991-12-30 00:00:00,Irrigation,12/30/1991,06/01,10/31,CARL SHIMEL
1,2,14950,36694,01,S013825,S013825_01,192.0,0.0,0.2,669954,Active,0.0,Diversion to Storage,Diversion point,Statement of Div and Use,Claimed,Point of Diversion,Gallons per Day,SHERBURNE,SLACK,Individual,67266.0,14950.0,S013825,01,S013825_01,8.0,S,4.0,W,4.0,,NE,NW,21.0,1926358.638086,6035514.543569,3.0,37.271356,-122.308314,,DD_NE,PESCADERO CREEK,N,N,SAN MATEO,San Mateo,,LA HONDA,J 041,7.5,,,9/29/2007 6:13:58,9.0,,1.805001e+11,18050006.0,San Francisco Coastal South,Lower Pescadero Creek,,2020-12-31 00:00:00,2020-01-01 00:00:00,1992-08-28 00:00:00,Domestic,08/28/1992,01/01,12/31,SHERBURNE SLACK
2,3,14951,2273,01,A010397,A010397_01,200.0,0.0,0.0,669955,Active,0.1,Diversion to Storage,Diversion point,Appropriative,Licensed,Point of Diversion,Gallons per Day,AL,RENKER,Individual,97853.0,14951.0,A010397,01,A010397_01,11.0,N,15.0,E,27.0,,NW,NE,21.0,2047785.971346,7050062.4187,2.0,38.772588,-120.286795,,DD_NE,SOUTH FORK AMERICAN RIVER,N,N,AMERICAN RIVER,El Dorado,,KYBURZ,G 061,7.5,012-220-03-01,,9/29/2007 6:14:05,9.0,DELTA WATERSHED,1.802013e+11,18020129.0,South Fork American,Chimney Flat-South Fork American River,,2020-11-01 00:00:00,2020-04-01 00:00:00,1942-03-11 00:00:00,Domestic,03/11/1942,04/01,11/01,AL RENKER


In [7]:
# Creating WaDE Custom water source native ID for easy water source identification
# ----------------------------------------------------------------------------------------------------

# Create temp WaterSourceNativeID dataframe of unique water source.
def assignWaterSourceNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "WaDECA_WS" + string1
    return outstring

dfWaterSourceNativeID = pd.DataFrame()
dfWaterSourceNativeID['in_WaterSourceName'] = dfPOD['SOURCE_NAME']
dfWaterSourceNativeID = dfWaterSourceNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfWaterSourceNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfWaterSourceNativeID['in_WaterSourceNativeID'] = dftemp.apply(lambda row: assignWaterSourceNativeID(row['Count']), axis=1)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
def retrieveWaterSourceNativeID(A):
    ml = dfWaterSourceNativeID.loc[(dfWaterSourceNativeID['in_WaterSourceName'] == A), 'in_WaterSourceNativeID']
    if not (ml.empty):  # check if the series is empty
        outList = ml.iloc[0]
    else:
        outList = ''
    return outList

dfPOD['in_WaterSourceNativeID'] = dfPOD.apply(lambda row: retrieveWaterSourceNativeID( row['SOURCE_NAME']), axis=1)
dfPOD.head(3)

,OID__x,CORE_POD_ID,WR_WATER_RIGHT_ID,POD_NUMBER,APPL_ID_x,APPL_POD_x,DIRECT_DIV_AMOUNT,DIVERSION_STORAGE_AMOUNT,DIVERSION_AC_FT,P_PLACE_ID,POD_STATUS,FACE_VALUE_AMOUNT,DIVERSION_TYPE,DIVERSION_CODE_TYPE,WR_TYPE,WR_STATUS,STORAGE_TYPE,POD_UNIT,FIRST_NAME,LAST_NAME,ENTITY_TYPE,OID__y,POD_ID,APPL_ID_y,POD_NUM,APPL_POD_y,TOWNSHIP_NUMBER,TOWNSHIP_DIRECTION,RANGE_NUMBER,RANGE_DIRECTION,SECTION_NUMBER,SECTION_CLASSIFIER,QUARTER,QUARTER_QUARTER,MERIDIAN,NORTH_COORD,EAST_COORD,SP_ZONE,LATITUDE,LONGITUDE,TRIB_DESC,LOCATION_METHOD,SOURCE_NAME,MOVEABLE,HAS_OPOD,WATERSHED,COUNTY,WELL_NUMBER,QUAD_MAP_NAME,QUAD_MAP_NUM,QUAD_MAP_MIN_SER,PARCEL_NUMBER,DIVERSION_SITE_NAME,LAST_UPDATE_DATE,LAST_UPDATE_USER_ID,SPECIAL_AREA,HUC_12,HUC_8,HU_8_NAME,HU_12_NAME,PRIORITY_DATE,DIRECT_DIV_SEASON_END,DIRECT_DIV_SEASON_START,APPLICATION_ACCEPTANCE_DATE,USE_CODE,in_AllocationPriorityDate,in_AllocationTimeframeStart,in_AllocationTimeframeEnd,in_WaDEOwner,in_WaterSourceNativeID
0,1,14949,36668,01,S013799,S013799_01,0.066,0.0,0.0,669953,Active,0.0,Diversion to Storage,Diversion point,Statement of Div and Use,Claimed,Point of Diversion,Cubic Feet per Second,CARL,SHIMEL,Individual,91156.0,14949.0,S013799,01,S013799_01,33.0,N,11.0,W,1.0,,,,21.0,2154480.547973,6265407.942305,1.0,40.739495,-123.069210,,DD_NE,HOCKER GULCH,N,Y,TRINITY RIVER,Trinity,,JUNCTION CITY,CC012,7.5,012-260-70,,9/29/2007 6:14:12,9.0,,1.801021e+11,18010211.0,Trinity,Conner Creek Trinity River,,2020-10-31 00:00:00,2020-06-01 00:00:00,1991-12-30 00:00:00,Irrigation,12/30/1991,06/01,10/31,CARL SHIMEL,WaDECA_WS1
1,2,14950,36694,01,S013825,S013825_01,192.0,0.0,0.2,669954,Active,0.0,Diversion to Storage,Diversion point,Statement of Div and Use,Claimed,Point of Diversion,Gallons per Day,SHERBURNE,SLACK,Individual,67266.0,14950.0,S013825,01,S013825_01,8.0,S,4.0,W,4.0,,NE,NW,21.0,1926358.638086,6035514.543569,3.0,37.271356,-122.308314,,DD_NE,PESCADERO CREEK,N,N,SAN MATEO,San Mateo,,LA HONDA,J 041,7.5,,,9/29/2007 6:13:58,9.0,,1.805001e+11,18050006.0,San Francisco Coastal South,Lower Pescadero Creek,,2020-12-31 00:00:00,2020-01-01 00:00:00,1992-08-28 00:00:00,Domestic,08/28/1992,01/01,12/31,SHERBURNE SLACK,WaDECA_WS2
2,3,14951,2273,01,A010397,A010397_01,200.0,0.0,0.0,669955,Active,0.1,Diversion to Storage,Diversion point,Appropriative,Licensed,Point of Diversion,Gallons per Day,AL,RENKER,Individual,97853.0,14951.0,A010397,01,A010397_01,11.0,N,15.0,E,27.0,,NW,NE,21.0,2047785.971346,7050062.4187,2.0,38.772588,-120.286795,,DD_NE,SOUTH FORK AMERICAN RIVER,N,N,AMERICAN RIVER,El Dorado,,KYBURZ,G 061,7.5,012-220-03-01,,9/29/2007 6:14:05,9.0,DELTA WATERSHED,1.802013e+11,18020129.0,South Fork American,Chimney Flat-South Fork American River,,2020-11-01 00:00:00,2020-04-01 00:00:00,1942-03-11 00:00:00,Domestic,03/11/1942,04/01,11/01,AL RENKER,WaDECA_WS3


## Export Outputs

In [8]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(dfPOD.dtypes)

OID__x                           int64
CORE_POD_ID                      int64
WR_WATER_RIGHT_ID                int64
POD_NUMBER                      object
APPL_ID_x                       object
APPL_POD_x                      object
DIRECT_DIV_AMOUNT               object
DIVERSION_STORAGE_AMOUNT        object
DIVERSION_AC_FT                 object
P_PLACE_ID                       int64
POD_STATUS                      object
FACE_VALUE_AMOUNT              float64
DIVERSION_TYPE                  object
DIVERSION_CODE_TYPE             object
WR_TYPE                         object
WR_STATUS                       object
STORAGE_TYPE                    object
POD_UNIT                        object
FIRST_NAME                      object
LAST_NAME                       object
ENTITY_TYPE                     object
OID__y                         float64
POD_ID                         float64
APPL_ID_y                       object
POD_NUM                         object
APPL_POD_y               

In [9]:
#Exporting to Finished File
dfPOD.to_csv('P_CaliforniaMaster.csv', index=False)  # The output